In [4]:
import boto3
import io
from PIL import Image
import s3fs
fs = s3fs.S3FileSystem()
s3 = boto3.client('s3', region_name='us-east-2')
import tensorflow as tf
import pandas as pd
from tensorflow.contrib.predictor import from_saved_model
import s3fs
import pandas as pd

In [5]:
def crop_save(s3, old_path:tuple, new_path:tuple, **kcrop):
    '''
    s3: reference of s3 client
    old_path = (old_bucket,old_key)
    new_path = (new_bucket,new_key)
    kcrop = {'l':rate0,'t':rate1,'r':rate2,'b':rate3}
    '''
    down_stream = s3.get_object(Bucket=old_path[0],Key=old_path[1])['Body']
    im = Image.open(down_stream)
    width, height = im.size
    left, top = width*kcrop['l'], height*kcrop['t']
    right, bottom = width*kcrop['r'], height*kcrop['b']
    im1 = im.crop((left, top, right, bottom))
    up_stream = io.BytesIO()
    im1.save(up_stream, format=im.format)
    up_stream.seek(0)
    s3.upload_fileobj(up_stream, new_path[0], new_path[1])
    down_stream.close()
    up_stream.close()
    return 

In [6]:
for old_path in fs.ls('s3:...')[1:]:
    pos = old_path.find('/')
    old_path = (old_path[:pos],old_path[pos+1:])
    k = old_path[1]
    p = k.rfind('/')
    new_path = ('Bucket Name','Object Name'+'cropped_'+k[p+1:])
    crop_save(s3, old_path, new_path, l=0.22,t=0.22,r=0.78,b=0.78)

In [4]:
df = fs.ls('s3:...')[1:]

In [5]:
df

['ingo-images/confusion-metrix/after-cropped/cropped_0.902_xdfp60-015_B03-3_BRITE_200223_232719_1.jpg',
 'ingo-images/confusion-metrix/after-cropped/cropped_0.90_xdfp60-009_C07-1_BRITE_200320_001838_1.jpg',
 'ingo-images/confusion-metrix/after-cropped/cropped_0.90_xdfp60-010_A02-1_BRITE_200202_235431_1.jpg',
 'ingo-images/confusion-metrix/after-cropped/cropped_0.90_xdfp60-010_A02-1_BRITE_200207_005813_1.jpg',
 'ingo-images/confusion-metrix/after-cropped/cropped_0.90_xdfp60-014_G05-3_BRITE_200208_205059_1.jpg',
 'ingo-images/confusion-metrix/after-cropped/cropped_0.9127_xdfp60-003_A01-1_BRITE_200124_004516_1.jpg',
 'ingo-images/confusion-metrix/after-cropped/cropped_0.91_xdfp60-001_C02-1_BRITE_200417_101735_1.jpg',
 'ingo-images/confusion-metrix/after-cropped/cropped_0.91_xdfp60-001_D11-2_BRITE_200119_013528_1.jpg',
 'ingo-images/confusion-metrix/after-cropped/cropped_0.91_xdfp60-010_A04-1_BRITE_200215_233601_1.jpg',
 'ingo-images/confusion-metrix/after-cropped/cropped_0.926_xdfp60-008_

In [7]:
model=from_saved_model("model")

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from model/variables/variables


In [8]:
my_list = [] #create my_list to append result
for file in fs.ls('s3://ingo-images/confusion-metrix/after-cropped/')[1:]:
    fp = fs.open('s3://' + file) #open file
    data = {'image_bytes': [fp.read()]} #read image
    results = model(data) #call the model
    final_result = results['classes'][0][0] #Slicing the first result
    if final_result.decode("utf-8").find('Crystal') != -1: #transfer the format of result
        my_list.append(str(1))
    else:
        my_list.append(str(0))

In [9]:
my_list

['1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0']

In [10]:
result_list = pd.DataFrame(my_list, columns = ['classify']) #transfer the list to dataframe

In [11]:
df = pd.DataFrame(df, columns = ['id']) #list the file
df_final = pd.concat([df, result_list], axis = 1) #concat two dataframes
df_final #final result

,id,classify
0,ingo-images/confusion-metrix/after-cropped/cro...,1
1,ingo-images/confusion-metrix/after-cropped/cro...,1
2,ingo-images/confusion-metrix/after-cropped/cro...,1
3,ingo-images/confusion-metrix/after-cropped/cro...,0
4,ingo-images/confusion-metrix/after-cropped/cro...,1
...,...,...
101,ingo-images/confusion-metrix/after-cropped/cro...,0
102,ingo-images/confusion-metrix/after-cropped/cro...,0
103,ingo-images/confusion-metrix/after-cropped/cro...,0
104,ingo-images/confusion-metrix/after-cropped/cro...,0


In [13]:
df_final.to_excel("df_final_final.xlsx")

In [12]:
df_final['classify'].value_counts()

0    70
1    36
Name: classify, dtype: int64

In [14]:
df_final_1 = df_final[df_final['classify']=='1']
df_final_1 = df_final_1.reset_index(drop = "True") # re-index
df_final_1

,id,classify
0,ingo-images/confusion-metrix/after-cropped/cro...,1
1,ingo-images/confusion-metrix/after-cropped/cro...,1
2,ingo-images/confusion-metrix/after-cropped/cro...,1
3,ingo-images/confusion-metrix/after-cropped/cro...,1
4,ingo-images/confusion-metrix/after-cropped/cro...,1
5,ingo-images/confusion-metrix/after-cropped/cro...,1
6,ingo-images/confusion-metrix/after-cropped/cro...,1
7,ingo-images/confusion-metrix/after-cropped/cro...,1
8,ingo-images/confusion-metrix/after-cropped/cro...,1
9,ingo-images/confusion-metrix/after-cropped/cro...,1


In [15]:
df_final_1[["id"]] = df_final_1[["id"]].astype('str')
a = df_final_1["id"]
a_frame = a.to_frame()
#a_frame
my_list_a = []
for i in a:
    fp = fs.open('s3://' + i)
    data = {'image_bytes': [fp.read()]} #read image
    results = model(data) 
    my_list_a.append(results['scores'][0][0])
my_list_a

[0.46713212,
 0.8907383,
 0.70967555,
 0.9152233,
 0.5927964,
 0.84674346,
 0.91294867,
 0.92617226,
 0.9035348,
 0.9287076,
 0.88332814,
 0.9240628,
 0.9207438,
 0.9168525,
 0.9064111,
 0.6265861,
 0.9256969,
 0.9180591,
 0.8581891,
 0.858901,
 0.90997934,
 0.72900707,
 0.9242982,
 0.62579453,
 0.6924942,
 0.6635391,
 0.5121309,
 0.89599,
 0.91966736,
 0.92552346,
 0.8523545,
 0.82376516,
 0.80981964,
 0.86629844,
 0.6162732,
 0.48452082]

In [16]:
my_list_a_df = pd.DataFrame(my_list_a, columns = ['scores'])
df_final_score = pd.concat([a_frame,my_list_a_df], axis = 1)
df_cropped = df_final_score.sort_values(by="scores",ascending=False)
df_cropped

,id,scores
9,ingo-images/confusion-metrix/after-cropped/cro...,0.928708
7,ingo-images/confusion-metrix/after-cropped/cro...,0.926172
16,ingo-images/confusion-metrix/after-cropped/cro...,0.925697
29,ingo-images/confusion-metrix/after-cropped/cro...,0.925523
22,ingo-images/confusion-metrix/after-cropped/cro...,0.924298
11,ingo-images/confusion-metrix/after-cropped/cro...,0.924063
12,ingo-images/confusion-metrix/after-cropped/cro...,0.920744
28,ingo-images/confusion-metrix/after-cropped/cro...,0.919667
17,ingo-images/confusion-metrix/after-cropped/cro...,0.918059
13,ingo-images/confusion-metrix/after-cropped/cro...,0.916852


In [17]:
df_cropped.to_excel("df_cropped.xlsx")

In [20]:
data = fs.ls('ingo-images/SMARCA2/crop_project/xdfp11-13-27/')

In [21]:
data[0:10]

['ingo-images/SMARCA2/crop_project/xdfp11-13-27/',
 'ingo-images/SMARCA2/crop_project/xdfp11-13-27/xdfp11-001_A01-1_BASE_171115_080723_1.jpg',
 'ingo-images/SMARCA2/crop_project/xdfp11-13-27/xdfp11-001_A01-1_BRITE_171114_190118_1.jpg',
 'ingo-images/SMARCA2/crop_project/xdfp11-13-27/xdfp11-001_A01-1_BRITE_171115_082055_1.jpg',
 'ingo-images/SMARCA2/crop_project/xdfp11-13-27/xdfp11-001_A01-1_BRITE_171115_095702_1.jpg',
 'ingo-images/SMARCA2/crop_project/xdfp11-13-27/xdfp11-001_A01-1_BRITE_171115_185822_1.jpg',
 'ingo-images/SMARCA2/crop_project/xdfp11-13-27/xdfp11-001_A01-1_BRITE_171116_075536_1.jpg',
 'ingo-images/SMARCA2/crop_project/xdfp11-13-27/xdfp11-001_A01-1_BRITE_171118_064603_1.jpg',
 'ingo-images/SMARCA2/crop_project/xdfp11-13-27/xdfp11-001_A01-1_BRITE_171121_064541_1.jpg',
 'ingo-images/SMARCA2/crop_project/xdfp11-13-27/xdfp11-001_A01-1_BRITE_171124_184642_1.jpg']